In [ ]:
import pandas as pd, numpy as np
dept_df = pd.read_csv('primary_df_2.csv').iloc[:,1:]
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()


In [ ]:
dept_df= dept_df.dropna(subset = ['Research Organizations - standardized'])
tmp_df = dept_df['Research Organizations - standardized'].astype(str).apply(lambda x: x.split(";"))
array_out = mlb.fit_transform(tmp_df)
df_out = pd.DataFrame(data=array_out, columns=mlb.classes_)
df_out.columns = df_out.columns.str.replace("'"," ")


In [ ]:
dept_df = dept_df.iloc[:,3:]
dept_df = dept_df.drop(['Country of Research organization','Times cited','Title','Title_abs'],axis=1)


In [ ]:
dept_df_2 = pd.concat([dept_df, df_out.set_index(dept_df.index)],axis =1).drop(['Research Organizations - standardized'],axis =1 )

In [ ]:
dept_df_2.to_csv('dept_df_2.csv',index_label=False)

In [ ]:
dept_df_2 = dept_df_2.fillna(0)
a = []
col_lst = np.append(a,(dept_df_2.iloc[:,2:].sum(axis=0)>100).values)
X = dept_df_2.iloc[:,np.argpartition(col_lst,-16)[-16:]]

In [ ]:
import pandas as pd, numpy as np
dept_df_2 = pd.read_csv('dept_df_2.csv')

In [ ]:
#dept_df_2 = pd.concat([dept_df['Authors at my Institution'], df_out],axis =1)


In [ ]:
#dept_df_2.to_csv('dept_df_2.csv')
#dept_df_2

In [ ]:
!pip install kmapper

     |████████████████████████████████| 125 kB 8.8 MB/s 


In [ ]:
import kmapper as km
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler

mapper = km.KeplerMapper(verbose=1)


projected_X = mapper.fit_transform(X,
                                   projection = TSNE(), scaler = None)
from sklearn import cluster



KeplerMapper(verbose=1)
..Composing projection pipeline of length 1:
	Projections: TSNE()
	Distance matrices: False
	Scalers: None
..Projecting on data shaped (3046, 16)

..Projecting data using: 
	TSNE(verbose=1)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 3046 samples in 0.000s...


/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


[t-SNE] Computed neighbors for 3046 samples in 0.402s...
[t-SNE] Computed conditional probabilities for sample 1000 / 3046
[t-SNE] Computed conditional probabilities for sample 2000 / 3046
[t-SNE] Computed conditional probabilities for sample 3000 / 3046
[t-SNE] Computed conditional probabilities for sample 3046 / 3046
[t-SNE] Mean sigma: 0.000000
[t-SNE] KL divergence after 250 iterations with early exaggeration: 113.781662
[t-SNE] KL divergence after 1000 iterations: 2.551870
Mapping on data shaped (3046, 2) using lens shaped (3046, 2)

Creating 625 hypercubes.
Merged 19 duplicate nodes.

Number of nodes before merger: 47; after merger: 28


Created 57 edges and 28 nodes in 0:00:00.156012.


In [ ]:
graph = mapper.map(projected_X,
                   clusterer=cluster.DBSCAN(eps=0.3, min_samples=5, metric = 'l1'),
                   cover=km.Cover(n_cubes=35, perc_overlap=0.6), remove_duplicate_nodes=True)


Mapping on data shaped (3046, 2) using lens shaped (3046, 2)

Creating 1225 hypercubes.
Merged 46 duplicate nodes.

Number of nodes before merger: 100; after merger: 54


Created 264 edges and 54 nodes in 0:00:00.451794.


In [ ]:
dept_df = pd.read_csv('primary_df_2.csv').iloc[:,1:]
dept_df['Departments'] = dept_df['Departments'].str[1:-1]

for i in range(dept_df.shape[0]):
  dept_df['Departments'][i] = dept_df['Departments'][i].replace("' ","'").replace("'","")



In [ ]:
Dept = dept_df.iloc[dept_df_2.index]['Departments'].astype(str).apply(lambda x: x.split(","))
Univ = dept_df.iloc[dept_df_2.index]['Research Organizations - standardized'].astype(str).apply(lambda x: x.split(";"))
html = mapper.visualize(graph,
                        path_html="Univ_collab.html",
                        title="Univ_Collab",
                        X_names = 'Dept- '+ Dept.astype(str) +'Univ' + Univ.astype(str) ,
                        color_function_name = ['Departments'],
                        include_searchbar = True,
                        custom_tooltips = ('Dept- '+ Dept.astype(str) +', Univ- ' + Univ.astype(str)).values )
from kmapper import jupyter
jupyter.display("Univ_collab.html")
km.draw_matplotlib(graph)
import matplotlib.pyplot as plt
plt.show()

/usr/local/lib/python3.7/dist-packages/kmapper/kmapper.py:867: UserWarning: `color_function_name` was set -- however, no `color_values` were passed, so default color_values were computed based on row order, and the passed `color_function_name` will be set as their label. This may be unexpected.
  "`color_function_name` was set -- however, no `color_values` were passed, so default color_values were computed based on row order, and the passed `color_function_name` will be set as their label. This may be unexpected."


Wrote visualization to: Univ_collab.html


no display found. Using non-interactive Agg backend
